# Environments

This document guides MAAP users in the process of selecting, extending existing environments (the set of libraries availables for analysis) or creating custom environments.

## Workspace environments

The MAAP ADE offers various workspace environment options that come with pre-installed essential libraries for basic computing and geospatial analysis. At the time of writing this guide, here are the options : 

![Workspace image options](../_static/workspace_options.png)

For example, the `MAAP RGEDI Stable` and `MAAP R Stable` workspace options come with various pre-installed R packages. 

**For more information** : Each of these options rely on Docker images that were build off from Dockerfiles that are publicly available in the [MAAP workspace repository](https://github.com/MAAP-Project/maap-workspaces/tree/develop). If you want to learn more about what libraries each image contains, check out this repository.

## Extending environments

Users may need libraries for their specific analysis purposes that are not present in the different environments offered. In this case, they will have to install these libraries themselves, and therefore select a package manager to do that.

### Base environment 

We recommend using the `mamba` package manager to extend environments (a fast `conda` drop-in replacement), using the `mamba` command line interface (CLI) pre-installed in all workspaces. In fact, the starting environment of all workspaces offered is a `mamba` environment. If you open a terminal launcher after creating a `Basic Stable` workspace : 


![Workspace image options](../_static/base_conda_environment_location.png)

You can notice that a `base` `mamba` environment is activated, and its libraries are located in `/opt/conda`. 

### Extending the `base` environment for a given workspace session

You can use the `mamba install` command to install additional packages in the `base` environment (run `mamba --help` to learn more about how to use `mamba` commands). For example :

```
mamba install xarray
```

_**Note**_ : when a workspace restarts, you get a fresh `base` environment and loose all the extensions you added. For this reason, it may be better to create your own custom `mamba` environment that you can preserve across sessions. 

## Custom environments

You can use the `mamba` CLI to create a new environment. 

The parameters (the list of libraries, the location where to search for them, etc...) can be passed either from a configuration file or directly on the console. We recommend using the first option (a YAML file is easier to share and modify). 

### Basic custom environment

Here is an example configuration file that we name `newenv` :

```
# newenv.yml
name: newenv
channels:
	- conda-forge
	- defaults
dependencies:
	- python=3.8
	- pandas=1.5.3
    - geopandas=0.12.2
```

It installs specific versions `python`, `pandas` and `geopandas` from either `conda-forge` or `defaults`. If versions aren't specified, the latest is installed. We recommend to always specify the version for reproducibility. The basic command to create this environment would be :

```
mamba env create -f newenv.yml
```

However, this stores this environment files in `/opt/conda`, which a directory that is reset when the workspace restarts, and so custom environments are lost. Therefore, you want to specify a storage location in your user directory with the `--prefix` parameter

```
mamba env create -f newenv.yml --prefix /projects/newenv
```

and to activate it : 

```
mamba activate newenv
```

### Using `pip` for python packages

Some python packages are not available via `mamba`. One can use `pip` within a `mamba` environment to download packages. The recommended way is to specify this in the configuration file. For example : 

```
name: newenv2
channels:
	- conda-forge
	- defaults
dependencies:
	- python=3.9
	- xarray=2023.3.0
	- pip=23.0.1
	- pip:
		- stackstac==0.4.3
```

### Using custom environments in jupyter notebooks

To make your environment accessible in a Jupyter notebook, you need first to have `ipykernel` listed as a dependency in your configuration file. Here is an example : 

```
name: newenv3
channels:
	- conda-forge
	- defaults
dependencies:
	- python=3.9
	- ipykernel=6.22.0
	- xarray=2023.3.0
	- pip=23.0.1
	- pip:
		- stackstac==0.4.3
```

Then, you register it with this command :

```
python -m ipykernel install --user --name newenv3 --display-name "Python newenv3"
```

Once this is done, wait 2-3 seconds and click on a the new launcher button. Among the notebook options, you should see 'Python newenv3'. This will spin up a notebook with your environment.
